### Analysis of Potential Model Features

In [ ]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from geopandas import GeoDataFrame
from shapely.geometry import Point

#import sys
import numpy as np
#import altair as alt
#import re
from random import choices
import datetime
from datetime import datetime

import matplotlib.pyplot as plt

import seaborn as sns

import boto3
import awswrangler

import re

import os
from os.path import isfile, join
from pathlib import Path
from os import listdir
import os
os.getcwd()

#from commons import download_data, find_vcs_root

path =  Path(os.getcwd())
root = path.parent.absolute()

root



In [ ]:
#heat_chart = X_test_20[[v, 'pred_4_weight_False', 'accident_count']].groupby(['collision_dayofweek','collision_hour']).agg({'pred_4_weight_False': ['mean']})

pvt_pred = pd.pivot_table(X_test_20, values='pred_8_weight_False', index=['collision_hour'],
                    columns=['collision_dayofweek'], aggfunc=np.mean)
pvt_actual = pd.pivot_table(X_test_20, values='actual', index=['collision_hour'],
                    columns=['collision_dayofweek'], aggfunc=np.mean)

pvt_pred.name = ['predicted']
pvt_actual.name = ['actual']

display(pvt.head(2))
#heat_chart.columns = ['collision_dayofweek','collision_hour', 'mean_prediction', 'mean_actual']

fig, ax = plt.subplots(1, 2, figsize=(8,10))

for i, d in enumerate([pvt_pred, pvt_actual]):
    sns.heatmap(d, 
            annot=False, 
            fmt="", 
            linewidth=1, 
            linecolor="#222",
            square=True,
            cmap='viridis_r',
            ax=ax[i],
               cbar_kws={"shrink": 0.25}).set(title=d.name[0])
plt.show()

In [ ]:
def univ_plot(data, var, predictor, cap = False, cap_threshold = 0):
    fig=plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(111)
    ax2 = ax1.twinx()
    
    data = data.copy()
    
    if cap:
        #print('capping')
        #print(cap_threshold)
        cap_mask = data[var] > cap_threshold
        data[var][cap_mask] = cap_threshold
    
    grp_df = data.groupby(var).agg({'actual': ['mean'], 
                                     predictor: ['mean', 'std', 'count'],
                                     #var: ['count']
                                   })
    #display(data[var].value_counts())
    grp_df.columns = ['actual', 'predicted', 'predicted_std', 'count']
    #grp_df['upper_bound'] = grp_df['predicted'] + grp_df['predicted_std']
    #grp_df['lower_bound'] = grp_df['predicted'] - grp_df['predicted_std']

    mean_actual = np.mean(data['actual'])
    mean_predicted = np.mean(data[predictor])
    max_actual = np.max(grp_df['actual'])
    max_predeicted = np.max(grp_df['predicted'])
    max_y = np.max([max_actual, max_predeicted])
    

    grp_df['predicted'] = grp_df['predicted'] * (mean_actual/mean_predicted)
    


    tall_df = pd.melt(grp_df.reset_index(), id_vars=var)  
    plt.tight_layout()
    
    #display(tall_df)
    
    count_mask = tall_df['variable'] == 'count'
    line_mask = tall_df['variable'].isin(['predicted','actual'])
    
    bar = sns.barplot(x=var,
                      y='value',
                      data=tall_df[count_mask],
                      color='gray',
                      ax=ax1)

    line1 = sns.lineplot(x=var,
                         y='value',
                         data=tall_df[line_mask],
                         marker='s',
                         hue='variable',
                         ax=ax2)
    
    handles, labels = ax2.get_legend_handles_labels()
    
    
    ax2.legend(handles=handles, labels=labels)
    ax2.set(ylim=(0, max_y*1.2))
    
    #line2 = sns.lineplot(x=grp_df.index,y='predicted',data=grp_df,marker='s',color='blue',ax=ax2)
    #plt.fill_between(x=grp_df.index, 
    #                 y1 = 'upper_bound',
    #                 y2 = 'lower_bound')
    bar.set_xticklabels(bar.get_xticklabels(), rotation=45, horizontalalignment='right')
    
    bar.set_xlabel("", fontsize = 12)
    
    #bar.set_title((var + ' Actual vs Predicted'), fontsize = 14, y = 1.15, x = 0.15)
    #bar.suptitle((var + ' Actual vs Predicted'), fontsize = 14, y = 1.05, x = 0.15)
    #plt.suptitle(title_string, y=1.05, fontsize=18)
    bar.set_ylabel("Frequency", fontsize = 12)
    line1.set_ylabel("Probability of Collision", fontsize = 12)
    
    title_string = (var + "\nPredicted vs Actual")
    print(title_string)
    #plt.suptitle(title_string, y=1.05, fontsize=18, loc='left')
    bar.set_title(title_string, fontsize=13, loc='left')
    #plt.legend()
    plt.show()

In [ ]:
#heat_chart = X_test_20[[v, 'pred_4_weight_False', 'accident_count']].groupby(['collision_dayofweek','collision_hour']).agg({'pred_4_weight_False': ['mean']})

pvt_pred = pd.pivot_table(X_test_20, values='pred_8_weight_False', index=['collision_hour'],
                    columns=['collision_dayofweek'], aggfunc=np.mean)
pvt_actual = pd.pivot_table(X_test_20, values='actual', index=['collision_hour'],
                    columns=['collision_dayofweek'], aggfunc=np.mean)

pvt_pred.name = ['predicted']
pvt_actual.name = ['actual']

display(pvt.head(2))
#heat_chart.columns = ['collision_dayofweek','collision_hour', 'mean_prediction', 'mean_actual']

fig, ax = plt.subplots(1, 2, figsize=(8,10))

for i, d in enumerate([pvt_pred, pvt_actual]):
    sns.heatmap(d, 
            annot=False, 
            fmt="", 
            linewidth=1, 
            linecolor="#222",
            square=True,
            cmap='viridis_r',
            ax=ax[i],
               cbar_kws={"shrink": 0.25}).set(title=d.name[0])
plt.show()

In [ ]:
univ_plot(data = X_test_20, var = 'neighbor1_prev_1_2', predictor = 'pred_8_weight_False',
         cap = True, cap_threshold = 50
         )